## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import os
import json
import string
import pickle
import numpy as np
import pandas as pd
from math import log
from judicial_splitter import splitter
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, KeyedVectors, Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim import matutils

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [3]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

# Базовая предобработка корпуса

Разделим корпус.

In [127]:
test_corpus = qa_corpus
questions, answers = {}, {}

for i, block in enumerate(test_corpus):
    questions[i] = block[0]
    answers[i] = block[1]  

Предобработаем вопросы и запишем их отдельно: целиком (для обратного индекса) и по параграфам (для остальных методов, для d2v - со стоп-словами и без), чтобы проще было оценивать качество.

In [129]:
for ind, text in questions.items():
    text = ' '.join(preprocessing(text, 'ii'))
    
    with open('questions/q_%s.txt' % (str(ind)), 'w', encoding='utf-8') as file:
        file.write(text)

In [130]:
for ind, text in questions.items():
    paragraphs = splitter(text, 4)
    
    with open('q_paragraphs/q_%s.txt' % (str(ind)), 'w', encoding='utf-8') as file:  
        for par in paragraphs:
            text = ' '.join(preprocessing(par, 'w2v'))
            file.write('%s\nend\n' % (text))

In [131]:
for ind, text in questions.items():
    paragraphs = splitter(text, 4)
    
    with open('q_paragraphs_with_stops/q_%s.txt' % (str(ind)), 'w', encoding='utf-8') as file:  
        for par in paragraphs:
            text = ' '.join(preprocessing(par, 'd2v'))
            file.write('%s\nend\n' % (text))

# Препроцессинг

In [132]:
def preprocessing(text, mode):
    morph = MorphAnalyzer()
    stops = stopwords.words('russian')
    punct = string.punctuation + '«»“”—…'
    
    text = text.lower()
    tokens = wordpunct_tokenize(text)
    all_lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    
    if mode == 'd2v':
        cleaned_lemmas = [lemma for lemma in all_lemmas if lemma not in punct]
    else:
        cleaned_lemmas = [lemma for lemma in all_lemmas if lemma not in stops and lemma not in punct]
 
    return cleaned_lemmas

# Обратный индекс

In [133]:
def corpus_and_stats(corpus):
    prep_corpus = []
    doc_lengths = {}
    
    for ind, text in corpus.items():
        text = preprocessing(text, 'ii')
        doc_lengths[ind] = len(text)

        prep_corpus.append(' '.join(text))
    
    avgdl = sum(doc_lengths.values()) / len(doc_lengths)
    
    return prep_corpus, doc_lengths, avgdl

In [134]:
def calc_idfs(term_doc_matrix):    
    idfs = {}

    docs = term_doc_matrix[0]
    words = term_doc_matrix[1]
    numbers = term_doc_matrix[2]

    N = len(docs)

    for i, word in enumerate(words):
        n = 0

        for num in numbers:
            val = num[i]

            if val != 0:
                n += 1

        idf = log((N - n + 0.5) / (n + 0.5))
        idfs[word] = idf
    
    return idfs

In [135]:
def inverted_index(corpus):
    prep_corpus, doc_lengths, avgdl = corpus_and_stats(corpus)
    
    vectorizer = CountVectorizer()
    docs = vectorizer.fit_transform(prep_corpus)
    index = [str(key) for key in corpus.keys()]

    term_doc_matrix = pd.DataFrame(docs.toarray(), columns=vectorizer.get_feature_names(), index=index)
    term_doc_matrix = [term_doc_matrix.index.tolist(), term_doc_matrix.columns.tolist(), term_doc_matrix.values.tolist()]
    
    idfs = calc_idfs(term_doc_matrix)
    
    return term_doc_matrix, doc_lengths, avgdl, idfs

Матрица и статистики

In [136]:
term_doc_matrix, doc_lengths, avgdl, idfs = inverted_index(answers)

Okapi B25

In [137]:
k1 = 2.0
b = 0.75

def score_BM25(idf, freq, D, avgdl) -> float:
    score = idf * (k1 + 1) * freq / (freq + k1 * (1 - b + b * D / avgdl))
    
    return score

Поиск

In [138]:
def search_inv_index(request):
    results = {doc: 0 for doc in term_doc_matrix[0]}
    
    for word in request:
        if word in term_doc_matrix[1]:
            word_ind = term_doc_matrix[1].index(word)
            
            for i, doc in enumerate(term_doc_matrix[0]):
                freq = term_doc_matrix[2][i][word_ind]
                D = doc_lengths[int(doc)]
                idf = idfs[word]      
                bm25 = score_BM25(idf, freq, D, avgdl)
                
                results[doc] += bm25
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=True)

# Word2Vec

In [13]:
model = Word2Vec.load('araneum_none_fasttextcbow_300_5_2018.model')

In [148]:
def get_w2v_vectors(paragraph):
    counter = 0
    hypervector = np.zeros(300)
    
    for word in paragraph:
        try:
            vector = np.array(model.wv[word])
            #print('vec', word, vector)
            hypervector += vector
            counter += 1
        except:
            continue

    #print(hypervector)

    hypervector = hypervector / counter
    
    return hypervector


def save_w2v_base(corpus):
    base = []

    for ind, text in corpus.items():
        paragraphs = splitter(text, 4)

        for par in paragraphs:
            prep_par = preprocessing(par, 'w2v')
            #print(prep_par)
            vector = get_w2v_vectors(prep_par)
            #print(vector.tolist())
            base.append([vector.tolist(), ind])
            
    with open('qa_base_w2v.json', 'w', encoding='utf-8') as file:
        json.dump(base, file, allow_nan=False)


In [149]:
save_w2v_base(answers)

Поиск

In [153]:
def search_w2v(request):
    vec_request = get_w2v_vectors(request)  
    #print(vec_request)
    results = {}
    
    with open('qa_base_w2v.json', 'r', encoding='utf-8') as file:
        base = json.load(file)
    
    for elem in base:
        #print(np.array(elem[0]))
        #print(np.isnan(vec_request), np.isnan(elem[0]))
        res = cosine_similarity([np.nan_to_num(vec_request)], [np.nan_to_num(elem[0])])[0][0]
        results[elem[1]] = res
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=True)

# Doc2Vec

In [2]:
class D2VIterator(object):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __iter__(self):
        for i, text in enumerate(self.texts):
              yield LabeledSentence(text, [self.labels[i]])

In [3]:
def train_doc2vec(d2v_iterator, model_name):
    model = Doc2Vec(vector_size=300, min_count=0, alpha=0.025, min_alpha=0.025)
    model.build_vocab(d2v_iterator)

    for epoch in range(100):
        print('epoch %s' % (epoch))
        model.train(d2v_iterator, total_examples=model.corpus_count, epochs=model.epochs)
        model.alpha -= 0.002
        model.min_alpha = model.alpha
        model.save(model_name)

In [4]:
def get_d2v_vectors(corpus, mode):
    prep_corpus, labels = [], []

    for ind, text in corpus.items():
        paragraphs = splitter(text, 4)

        for i, par in enumerate(paragraphs):
            if mode == 'with_stops':
                prep_par = preprocessing(par, 'd2v')
            elif mode == 'without_stops':
                prep_par = preprocessing(par, 'w2v')
                
            prep_corpus.append(' '.join(text))
            labels.append(ind)
        
    d2v_iterator = D2VIterator(prep_corpus, labels)
    train_doc2vec(d2v_iterator, 'qa_doc2vec.model')
    trained_model = Doc2Vec.load('qa_doc2vec.model')
    
    return trained_model, labels


def save_d2v_base(corpus, mode):
    trained_model, labels = get_d2v_vectors(corpus, mode)
    base = []
    
    for label in labels:
        vector = trained_model.docvecs[label]
        base.append([vector.tolist(), label])
        
    with open('qa_base_d2v_%s.json' % (mode), 'w', encoding='utf-8') as file:
        json.dump(base, file)
        
    return trained_model

In [166]:
trained_model_no_stops = save_d2v_base(answers, 'without_stops')

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99


In [167]:
trained_model_stops = save_d2v_base(answers, 'with_stops')

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99


Поиск

In [9]:
def search_d2v(request, mode):
    trained_model_no_stops = Doc2Vec.load('qa_doc2vec.model')
    vec_request = trained_model_no_stops.infer_vector(request)
        
    results = {}
    
    with open('qa_base_d2v_%s.json' % (mode), 'r', encoding='utf-8') as file:
        base = json.load(file)
    
    for elem in base:
        res = cosine_similarity([np.nan_to_num(vec_request)], [np.nan_to_num(elem[0])])
        results[elem[1]] = res    
    
    return sorted(results.items(), key=lambda kv: kv[1], reverse=True)

# Оценка

In [5]:
def evaluate_results(folder, mode, d2v_mode=None):
    files = os.listdir(folder)
    score = 0
    
    for file in files:
        num = file.strip('q_').strip('.txt') 
        file = folder + '/' + file
            
        if mode == 'inverted_index':
            with open(file, 'r', encoding='utf-8') as file:
                request = file.read().split(' ')
            
            top_results = search_inv_index(request)[:5]
            
        elif mode == 'word2vec' or mode == 'doc2vec':
            with open(file, 'r', encoding='utf-8') as file:
                paragraphs = file.read().split('\nend\n')
                
            top_results = []
            
            for par in paragraphs:
                par = par.split(' ')
                if mode == 'word2vec':
                    cur_results = search_w2v(par)
                elif mode == 'doc2vec':
                    cur_results = search_d2v(par, d2v_mode)
                    
                top_results.extend(cur_results)
             
            top_results = top_results[:5]
        
        else:
            raise TypeError('unsupported search method')
        #print(num, top_results)        
        for res in top_results:
            if str(res[0]) == str(num):
                score += 1
                break
        
    return score / len(files)

Обратный индекс

In [170]:
evaluate_results('questions', 'inverted_index')

0.5643063583815029

In [171]:
evaluate_results('questions', 'word2vec')

0.24855491329479767

Word2Vec

In [172]:
evaluate_results('q_paragraphs', 'word2vec')

0.23482658959537572

Doc2Vec без стоп-слов.

In [10]:
evaluate_results('q_paragraphs', 'doc2vec', 'without_stops')

0.0036127167630057803